### Use saprQL to query DBpedia and merge into knowledge graph

In [4]:
import rdflib
g=rdflib.Graph()
g.load('http://dbpedia.org/resource/Biomedical')

for s,p,o in g:
    print (s,p,o)

http://dbpedia.org/resource/Biomedical http://dbpedia.org/ontology/wikiPageRevisionID 288855867
http://dbpedia.org/resource/Biomedical http://www.w3.org/2000/01/rdf-schema#label Biomedical
http://dbpedia.org/resource/Biomedical http://dbpedia.org/ontology/wikiPageID 609268
http://dbpedia.org/resource/Biomedical http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Biomedical
http://dbpedia.org/resource/Biomedical http://www.w3.org/ns/prov#wasDerivedFrom http://en.wikipedia.org/wiki/Biomedical?oldid=288855867
http://en.wikipedia.org/wiki/Biomedical http://xmlns.com/foaf/0.1/primaryTopic http://dbpedia.org/resource/Biomedical
http://dbpedia.org/resource/Charles_Stark_Draper_Laboratory http://dbpedia.org/ontology/industry http://dbpedia.org/resource/Biomedical
http://dbpedia.org/resource/Biomedical http://dbpedia.org/ontology/wikiPageRedirects http://dbpedia.org/resource/Medical_research
http://dbpedia.org/resource/Musa_Hakan_Asyalı http://dbpedia.org/ontology/knownFor http://d

In [20]:
from SPARQLWrapper import SPARQLWrapper, JSON

prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpedia: <http://dbpedia.org/resource/>PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>PREFIX category: <http://dbpedia.org/resource/Category:>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>PREFIX foaf: <http://xmlns.com/foaf/0.1/>PREFIX dbpprop: <http://dbpedia.org/property/>
    PREFIX dbprop: <http://dbpedia.org/property/>PREFIX grs: <http://www.georss.org/georss/>
    PREFIX category: <http://dbpedia.org/resource/Category:>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX freebase: <http://rdf.freebase.com/ns/>
    PREFIX db: <http://dbpedia.org/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX http: <http://www.w3.org/2006/http#>"""

In [21]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(prefix + """    
    SELECT ?z
    WHERE { <http://dbpedia.org/resource/Biomedical_engineering> rdf:type ?z }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result)

{'z': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#Thing'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/ontology/Software'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/class/yago/Abstraction100002137'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/class/yago/Cognition100023271'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/class/yago/Content105809192'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/class/yago/Discipline105996646'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/class/yago/KnowledgeDomain105999266'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/class/yago/PsychologicalFeature100023100'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/class/yago/WikicatEngineeringDisciplines'}}


In [22]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(prefix + """
    SELECT ?z
    WHERE { <http://dbpedia.org/resource/Biomedical_engineering> rdfs:label ?z }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    if result['z']['xml:lang'] == 'en':
        print(result['z']['value'])

Biomedical engineering
